In [1]:
## Lets import our modules
import boto3
import pandas as pd
#import psycopg2
import pymysql
from sqlalchemy import create_engine
import os
import sqlite3
import sys
parentDirectory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0,parentDirectory)
from secretsPaul import secrets
import mysql.connector
import plotly.express as px
from collections import Counter

In [2]:
#connections
#def create_connection():
endpoint = 'drown-murphy-lankheet.cluster-ceswqg9qwa9i.us-east-1.rds.amazonaws.com'#secrets.get('DATABASE_ENDPOINT')
user = secrets.get('DATABASE_USER')
password = secrets.get('DATABASE_PASSWORD')
port=secrets.get('DATABASE_PORT')
connection = create_engine(f'mysql+pymysql://{user}:{password}@{endpoint}:{port}/Capstone', pool_recycle=3600,  connect_args={'connect_timeout': 10});
    
connect=mysql.connector.connect(host=endpoint, database='Capstone', user=user, password=password)
cursor = connect.cursor()
    #return cursor
#cursor=create_connection()

In [3]:
#Pulling in listings
all_listings = pd.read_sql('Select scrape_city, amenities, review_scores_value  from recent_listings_occup',connection) 

In [21]:
## NLP FUNCTIONS - Updated to handle looping

def get_top_amenities(listing_df):
    # correlating amenities with review scores
    # adapted from https://stackoverflow.com/questions/48873233/is-there-a-way-to-get-correlation-with-string-data-and-a-numerical-value-in-pand

    df = listing_df.copy()
    amen_list = list(df['amenities'])

    new_list = [item.strip('[]\' ""').replace(' u2013 ', ': ').replace('u2019', "'").replace('"', '') for item in
                amen_list]

    list_of_lists = []
    for item in new_list:
        new_item = item.split(",")
        list_of_lists.append(new_item)

    flattened = [val for sublist in list_of_lists for val in sublist]
    counts = Counter(flattened)

    high_counts = {k: c for k, c in counts.items() if c >= 500}

    top_amen = list(high_counts.keys())

    full_new_list = []
    for each_list in list_of_lists:
        new_new_list = list(set(top_amen).intersection(set(each_list)))
        full_new_list.append(new_new_list)

    df['amenities'] = full_new_list

    s_corr = pd.DataFrame(df.amenities.str.get_dummies(sep=',').corrwith(
        df.review_scores_value / df.review_scores_value.max()))
    s_corr.reset_index(inplace=True)
    s_corr.rename(columns={'index': 'amenity', 0: 'score'}, inplace=True)

    for idx, item in enumerate(s_corr['amenity']):
        s_corr.at[idx, 'amenity'] = s_corr.at[idx, 'amenity'].lower().strip('[]\' ""')
    s_corr = s_corr.sort_values(by='score', ascending=False)
    s_corr['score'] = s_corr['score'].round(3)

    amenities_df = s_corr.set_index('amenity')[:20]

    return amenities_df

In [19]:
#unique list of all cities
scrape_list=list(all_listings['scrape_city'].unique())

In [27]:
#Looping through each city and running the amenities
top_amenities=pd.DataFrame(columns=['amenity','score','scrape_city'])
for city in scrape_list:
    topa=get_top_amenities(all_listings[all_listings['scrape_city']==city])
    topa['scrape_city']=city
    topa.reset_index(inplace=True)
    top_amenities=pd.concat([top_amenities, topa],ignore_index=True)
    

In [29]:
top_amenities.to_sql('top_amenities',con=connection, if_exists='replace',index=False, chunksize=1000, method='multi')

582